In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import os

## Initialising params

In [12]:
lr_ = 1e-4
epochs_ = 20
batchSize = 32

## Loading Image

In [4]:
imagePaths = list(paths.list_images('data'))
data = []
labels = []
#go over image paths
for imagePath in imagePaths:
    #labels
    label = imagePath.split(os.path.sep)[-2]
    #loading img and processing 
    image = load_img(imagePath, target_size=(224, 224))
    image = img_to_array(image)
    image = preprocess_input(image)
    #appending stuff
    data.append(image)
    labels.append(label)
# np arrays bro
data = np.array(data, dtype="float32")
labels = np.array(labels)

In [5]:
data

array([[[[-0.94509804, -0.9764706 , -0.85882354],
         [-0.94509804, -0.9764706 , -0.85882354],
         [-0.94509804, -0.9764706 , -0.85882354],
         ...,
         [-0.8509804 , -0.90588236, -0.78039217],
         [-0.8509804 , -0.90588236, -0.78039217],
         [-0.84313726, -0.8980392 , -0.77254903]],

        [[-0.9137255 , -0.94509804, -0.827451  ],
         [-0.9137255 , -0.94509804, -0.827451  ],
         [-0.9137255 , -0.94509804, -0.827451  ],
         ...,
         [-0.7647059 , -0.79607844, -0.6784314 ],
         [-0.7647059 , -0.79607844, -0.6784314 ],
         [-0.75686276, -0.7882353 , -0.67058825]],

        [[-0.92156863, -0.9607843 , -0.8039216 ],
         [-0.92156863, -0.9607843 , -0.8039216 ],
         [-0.92156863, -0.9607843 , -0.8039216 ],
         ...,
         [-0.88235295, -0.8980392 , -0.7882353 ],
         [-0.8666667 , -0.88235295, -0.77254903],
         [-0.85882354, -0.8745098 , -0.7647059 ]],

        ...,

        [[ 0.3803922 , -0.9607843 , -0

In [6]:
labels

array(['without_mask', 'without_mask', 'without_mask', ..., 'with_mask',
       'with_mask', 'with_mask'], dtype='<U12')

## Converting labels to binary cat. vars.

In [7]:
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)

## train test split

In [8]:
(trainX, testX, trainY, testY) = train_test_split(data, labels,test_size=0.20, stratify=labels, random_state=42)

## image augmentation

In [9]:
aug = ImageDataGenerator(rotation_range=20,zoom_range=0.15,width_shift_range=0.2,height_shift_range=0.2,shear_range=0.15,horizontal_flip=True,fill_mode="nearest")

In [10]:
labels

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)

## imagenet model

In [11]:
baseModel = MobileNetV2(weights="imagenet", include_top=False,
	input_tensor=Input(shape=(224, 224, 3)))

#attaching a custom fine tuned top to the base model

headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(2, activation="softmax")(headModel)

#final model used for training
model = Model(inputs=baseModel.input, outputs=headModel)

#making base model layers non trainable
for layer in baseModel.layers:
    layer.trainable = False

C:\Users\manavgakhar\anaconda3\lib\site-packages\keras_applications\mobilenet_v2.py:294: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


9412608/9406464 [==============================] - 74s 8us/step


## compiling model

In [13]:
opt = Adam(lr=lr_, decay=lr_ /epochs_)
model.compile(loss="binary_crossentropy", optimizer=opt,
    metrics=["accuracy"])

## training

In [15]:
H = model.fit(
    aug.flow(trainX, trainY, batch_size=batchSize),
    steps_per_epoch=len(trainX) // batchSize,
    validation_data=(testX, testY),
    validation_steps=len(testX) // batchSize,
    epochs=epochs_)

  ...
    to  
  ['...']
Train for 34 steps, validate on 276 samples
Epoch 1/20
34/34 [==============================] - 12s 344ms/step - loss: 0.5853 - accuracy: 0.6910 - val_loss: 0.3169 - val_accuracy: 0.9180
Epoch 2/20
34/34 [==============================] - 9s 279ms/step - loss: 0.3680 - accuracy: 0.8483 - val_loss: 0.1993 - val_accuracy: 0.9531
Epoch 3/20
34/34 [==============================] - 9s 276ms/step - loss: 0.2538 - accuracy: 0.9036 - val_loss: 0.1274 - val_accuracy: 0.9844
Epoch 4/20
34/34 [==============================] - 10s 282ms/step - loss: 0.2088 - accuracy: 0.9204 - val_loss: 0.1005 - val_accuracy: 0.9844
Epoch 5/20
34/34 [==============================] - 9s 267ms/step - loss: 0.1862 - accuracy: 0.9335 - val_loss: 0.0905 - val_accuracy: 0.9766
Epoch 6/20
34/34 [==============================] - 9s 263ms/step - loss: 0.1506 - accuracy: 0.9429 - val_loss: 0.0722 - val_accuracy: 0.9844
Epoch 7/20
34/34 [==============================] - 9s 279ms/step - loss: 0.1

In [16]:
predidx = model.predict(testX, batch_size=batchSize)

# Finding args with max prob
predidx = np.argmax(predidx, axis=1)

print(classification_report(testY.argmax(axis=1), predidx,target_names=lb.classes_))

              precision    recall  f1-score   support

   with_mask       0.97      1.00      0.99       138
without_mask       1.00      0.97      0.99       138

    accuracy                           0.99       276
   macro avg       0.99      0.99      0.99       276
weighted avg       0.99      0.99      0.99       276



In [19]:
#saving model lol
model.save("model", save_format="h5")